In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
%sql
USE CATALOG medalhao;
SHOW TABLES IN medalhao.silver;

database,tableName,isTemporary
silver,dm_categoria_produtos_traducao,false
silver,dm_cotacao_dolar,false
silver,ft_avaliacoes_pedidos,false
silver,ft_consumidores,false
silver,ft_itens_pedidos,false
silver,ft_pagamentos_pedidos,false
silver,ft_pedido_total,false
silver,ft_pedidos,false
silver,ft_produtos,false
silver,ft_vendedores,false


## 1. Área de Logística

### 1.1 `gold.ft_vendas_consumidor_local`

In [0]:
df_pedido_total = spark.table('silver.ft_pedido_total')
df_consumidores = spark.table('silver.ft_consumidores')
df_produtos = spark.table('silver.ft_produtos')
df_pedido_total.printSchema()
df_consumidores.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- id_consumidor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- valor_total_pago_brl: double (nullable = true)
 |-- valor_total_pago_usd: double (nullable = true)
 |-- data_pedido: date (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

root
 |-- id_consumidor: string (nullable = true)
 |-- prefixo_cep: integer (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



In [0]:
df_temp = df_pedido_total.join(
    df_consumidores,
    on = 'id_consumidor',
    how = "left"
)
display(df_temp.printSchema())

root
 |-- id_consumidor: string (nullable = true)
 |-- id_pedido: string (nullable = true)
 |-- status: string (nullable = true)
 |-- valor_total_pago_brl: double (nullable = true)
 |-- valor_total_pago_usd: double (nullable = true)
 |-- data_pedido: date (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)
 |-- prefixo_cep: integer (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



In [0]:
df_gold_ft_vendas_consumidor_local = (df_temp.select(
    F.col("id_pedido"),
    F.col("id_consumidor"),
    F.col("valor_total_pago_brl"),
    F.col("cidade"),
    F.col("estado"),
    F.col("data_pedido")
) .withColumn("data_ingestao", F.current_timestamp())
)
df_gold_ft_vendas_consumidor_local.write \
.format("delta") \
.mode("overwrite") \
.saveAsTable('gold.ft_vendas_consumidor_local') 

In [0]:
display(df_gold_ft_vendas_consumidor_local.limit(5))

id_pedido,id_consumidor,valor_total_pago_brl,cidade,estado,data_pedido,data_ingestao
00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,72.19,CAMPOS DOS GOYTACAZES,RJ,2017-09-13,2025-11-19T13:17:08.990Z
00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,259.83,SANTA FE DO SUL,SP,2017-04-26,2025-11-19T13:17:08.990Z
000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,216.87,PARA DE MINAS,MG,2018-01-14,2025-11-19T13:17:08.990Z
00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,25.78,ATIBAIA,SP,2018-08-08,2025-11-19T13:17:08.990Z
00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,218.04,VARZEA PAULISTA,SP,2017-02-04,2025-11-19T13:17:08.990Z


### 1.2 Criação da view `gold.view_total_compras_por_consumidor`

In [0]:
view_name = "gold.view_total_compras_por_consumidor"
source_table_name = "gold.ft_vendas_consumidor_local"
df_source = spark.table(source_table_name)
df_view_vendas_localidade = (
    df_source
    .groupBy("cidade", "estado")
    .agg(
        F.round(F.sum("valor_total_pago_brl"), 2).alias("valor_total_localidade"),
        F.count("id_pedido").alias("quantidade_vendas")
    )
)
df_view_vendas_localidade.createOrReplaceTempView("gold.view_total_compras_por_consumidor")

In [0]:
df_view_vendas_localidade.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(view_name)

In [0]:
display(df_view_vendas_localidade.limit(5))

cidade,estado,valor_total_localidade,quantidade_vendas
CAMPOS DOS GOYTACAZES,RJ,40212.11,238
SANTA FE DO SUL,SP,4668.63,35
PARA DE MINAS,MG,7994.59,37
ATIBAIA,SP,24181.1,159
VARZEA PAULISTA,SP,12789.56,80


Crie uma consulta SQL para exibir o total de vendas por estado

In [0]:
spark.sql("SELECT estado, SUM(quantidade_vendas) as total_vendas, SUM(valor_total_localidade) as valor_total FROM gold.view_total_compras_por_consumidor GROUP BY estado ORDER BY total_vendas").show()

+------+------------+------------------+
|estado|total_vendas|       valor_total|
+------+------------+------------------+
|    RR|          46|          10064.62|
|    AP|          68|16262.800000000001|
|    AC|          81|          19680.62|
|    AM|         148|          27966.93|
|    RO|         253|           60866.2|
|    TO|         280|61485.329999999994|
|    SE|         350|          75246.25|
|    AL|         413| 96962.06000000001|
|    RN|         485|102718.12999999999|
|    PI|         495|         108523.97|
|    PB|         536|         141545.72|
|    MS|         715|137534.84000000003|
|    MA|         747|152523.02000000005|
|    MT|         907|187029.29000000004|
|    PA|         975|218295.84999999998|
|    CE|        1336|         279464.03|
|    PE|        1652|         324850.44|
|    GO|        2020|350092.30999999994|
|    ES|        2033|325967.55000000016|
|    DF|        2140| 355141.0800000001|
+------+------------+------------------+
only showing top

## 2. Área de Logística

### 2.1 Criação da tabela `gold.ft_atrasos_pedidos_local_vendedor`

In [0]:
df_pedidos = spark.table('silver.ft_pedidos')
df_itens_pedidos = spark.table('silver.ft_itens_pedidos')
df_pedidos.printSchema()
df_itens_pedidos.printSchema() 

root
 |-- id_pedido: string (nullable = true)
 |-- id_consumidor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- pedido_compra_timestamp: date (nullable = true)
 |-- pedido_aprovado_timestamp: date (nullable = true)
 |-- pedido_carregado_timestamp: date (nullable = true)
 |-- pedido_entregue_timestamp: date (nullable = true)
 |-- pedido_estimativa_entrega_timestamp: date (nullable = true)
 |-- tempo_entrega_dias: integer (nullable = true)
 |-- tempo_entrega_estimado_dias: integer (nullable = true)
 |-- diferenca_entrega_dias: integer (nullable = true)
 |-- entrega_no_prazo: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

root
 |-- id_pedido: string (nullable = true)
 |-- id_item: integer (nullable = true)
 |-- id_produto: string (nullable = true)
 |-- id_vendedor: string (nullable = true)
 |-- preco_BRL: double (nullable = true)
 |-- preco_frete: double (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



In [0]:
df_ft_vendedor = df_itens_pedidos.select("id_pedido", "id_vendedor")
display(df_ft_vendedor.limit(10))

id_pedido,id_vendedor
00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202
00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36
000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d
00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4
00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87
00048cc3ae777c65dbb7d2a0634bc1ea,6426d21aca402a131fc0a5d0960a3c90
00054e8431b9d7675808bcb819fb4a32,7040e82f899a04d1b434b795a43b4617
000576fe39319847cbb9d288c5617fa6,5996cddab893a4652a15592fb58ab8db
0005a1a1728c9d785b8e2b08b904576c,a416b6a846a11724393025641d4edd5e
0005f50442cb953dcd1d21e1fb923495,ba143b05f0110f0dc71ad71b4466ce92


In [0]:
df_join = df_pedidos.join(df_consumidores, on = "id_consumidor", how = "left")
display(df_join.limit(10))
df_join.printSchema()

id_consumidor,id_pedido,status,pedido_compra_timestamp,pedido_aprovado_timestamp,pedido_carregado_timestamp,pedido_entregue_timestamp,pedido_estimativa_entrega_timestamp,tempo_entrega_dias,tempo_entrega_estimado_dias,diferenca_entrega_dias,entrega_no_prazo,data_ingestao,prefixo_cep,cidade,estado,data_ingestao
3ce436f183e68e07877b285a838db11a,00010242fe8c5a6d1ba2dd792cb16214,entregue,2017-09-13,2017-09-13,2017-09-19,2017-09-20,2017-09-29,7,16,-9,Sim,2025-11-13T00:17:18.905Z,28013,CAMPOS DOS GOYTACAZES,RJ,2025-11-13T01:31:27.642Z
f6dd3ec061db4e3987629fe6b26e5cce,00018f77f2f0320c557190d7a144bdd3,entregue,2017-04-26,2017-04-26,2017-05-04,2017-05-12,2017-05-15,16,19,-3,Sim,2025-11-13T00:17:18.905Z,15775,SANTA FE DO SUL,SP,2025-11-13T01:31:27.642Z
6489ae5e4333f3693df5ad4372dab6d3,000229ec398224ef6ca0657da4fc703e,entregue,2018-01-14,2018-01-14,2018-01-16,2018-01-22,2018-02-05,8,22,-14,Sim,2025-11-13T00:17:18.905Z,35661,PARA DE MINAS,MG,2025-11-13T01:31:27.642Z
d4eb9395c8c0431ee92fce09860c5a06,00024acbcdf0a6daa1e931b038114c75,entregue,2018-08-08,2018-08-08,2018-08-10,2018-08-14,2018-08-20,6,12,-6,Sim,2025-11-13T00:17:18.905Z,12952,ATIBAIA,SP,2025-11-13T01:31:27.642Z
58dbd0b2d70206bf40e62cd34e84d795,00042b26cf59d7ce69dfabb4e55b4fd9,entregue,2017-02-04,2017-02-04,2017-02-16,2017-03-01,2017-03-17,25,41,-16,Sim,2025-11-13T00:17:18.905Z,13226,VARZEA PAULISTA,SP,2025-11-13T01:31:27.642Z
816cbea969fe5b689b39cfc97a506742,00048cc3ae777c65dbb7d2a0634bc1ea,entregue,2017-05-15,2017-05-17,2017-05-17,2017-05-22,2017-06-06,7,22,-15,Sim,2025-11-13T00:17:18.905Z,38017,UBERABA,MG,2025-11-13T01:31:27.642Z
32e2e6ab09e778d99bf2e0ecd4898718,00054e8431b9d7675808bcb819fb4a32,entregue,2017-12-10,2017-12-10,2017-12-12,2017-12-18,2018-01-04,8,25,-17,Sim,2025-11-13T00:17:18.905Z,16700,GUARARAPES,SP,2025-11-13T01:31:27.642Z
9ed5e522dd9dd85b4af4a077526d8117,000576fe39319847cbb9d288c5617fa6,entregue,2018-07-04,2018-07-05,2018-07-05,2018-07-09,2018-07-25,5,21,-16,Sim,2025-11-13T00:17:18.905Z,11702,PRAIA GRANDE,SP,2025-11-13T01:31:27.642Z
16150771dfd4776261284213b89c304e,0005a1a1728c9d785b8e2b08b904576c,entregue,2018-03-19,2018-03-20,2018-03-28,2018-03-29,2018-03-29,10,10,0,Sim,2025-11-13T00:17:18.905Z,11075,SANTOS,SP,2025-11-13T01:31:27.642Z
351d3cb2cee3c7fd0af6616c82df21d3,0005f50442cb953dcd1d21e1fb923495,entregue,2018-07-02,2018-07-02,2018-07-03,2018-07-04,2018-07-23,2,21,-19,Sim,2025-11-13T00:17:18.905Z,6636,JANDIRA,SP,2025-11-13T01:31:27.642Z


root
 |-- id_consumidor: string (nullable = true)
 |-- id_pedido: string (nullable = true)
 |-- status: string (nullable = true)
 |-- pedido_compra_timestamp: date (nullable = true)
 |-- pedido_aprovado_timestamp: date (nullable = true)
 |-- pedido_carregado_timestamp: date (nullable = true)
 |-- pedido_entregue_timestamp: date (nullable = true)
 |-- pedido_estimativa_entrega_timestamp: date (nullable = true)
 |-- tempo_entrega_dias: integer (nullable = true)
 |-- tempo_entrega_estimado_dias: integer (nullable = true)
 |-- diferenca_entrega_dias: integer (nullable = true)
 |-- entrega_no_prazo: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)
 |-- prefixo_cep: integer (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)



In [0]:
df_join_1 = df_join.join(df_ft_vendedor, on="id_pedido", how="left")
display(df_join_1.limit(10))
df_join_1.printSchema()

id_pedido,id_consumidor,status,pedido_compra_timestamp,pedido_aprovado_timestamp,pedido_carregado_timestamp,pedido_entregue_timestamp,pedido_estimativa_entrega_timestamp,tempo_entrega_dias,tempo_entrega_estimado_dias,diferenca_entrega_dias,entrega_no_prazo,data_ingestao,prefixo_cep,cidade,estado,data_ingestao,id_vendedor
00054e8431b9d7675808bcb819fb4a32,32e2e6ab09e778d99bf2e0ecd4898718,entregue,2017-12-10,2017-12-10,2017-12-12,2017-12-18,2018-01-04,8,25,-17,Sim,2025-11-13T00:17:18.905Z,16700,GUARARAPES,SP,2025-11-13T01:31:27.642Z,7040e82f899a04d1b434b795a43b4617
00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,entregue,2018-08-08,2018-08-08,2018-08-10,2018-08-14,2018-08-20,6,12,-6,Sim,2025-11-13T00:17:18.905Z,12952,ATIBAIA,SP,2025-11-13T01:31:27.642Z,9d7a1d34a5052409006425275ba1c2b4
00048cc3ae777c65dbb7d2a0634bc1ea,816cbea969fe5b689b39cfc97a506742,entregue,2017-05-15,2017-05-17,2017-05-17,2017-05-22,2017-06-06,7,22,-15,Sim,2025-11-13T00:17:18.905Z,38017,UBERABA,MG,2025-11-13T01:31:27.642Z,6426d21aca402a131fc0a5d0960a3c90
00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,entregue,2017-09-13,2017-09-13,2017-09-19,2017-09-20,2017-09-29,7,16,-9,Sim,2025-11-13T00:17:18.905Z,28013,CAMPOS DOS GOYTACAZES,RJ,2025-11-13T01:31:27.642Z,48436dade18ac8b2bce089ec2a041202
0005f50442cb953dcd1d21e1fb923495,351d3cb2cee3c7fd0af6616c82df21d3,entregue,2018-07-02,2018-07-02,2018-07-03,2018-07-04,2018-07-23,2,21,-19,Sim,2025-11-13T00:17:18.905Z,6636,JANDIRA,SP,2025-11-13T01:31:27.642Z,ba143b05f0110f0dc71ad71b4466ce92
000576fe39319847cbb9d288c5617fa6,9ed5e522dd9dd85b4af4a077526d8117,entregue,2018-07-04,2018-07-05,2018-07-05,2018-07-09,2018-07-25,5,21,-16,Sim,2025-11-13T00:17:18.905Z,11702,PRAIA GRANDE,SP,2025-11-13T01:31:27.642Z,5996cddab893a4652a15592fb58ab8db
000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,entregue,2018-01-14,2018-01-14,2018-01-16,2018-01-22,2018-02-05,8,22,-14,Sim,2025-11-13T00:17:18.905Z,35661,PARA DE MINAS,MG,2025-11-13T01:31:27.642Z,5b51032eddd242adc84c38acab88f23d
0005a1a1728c9d785b8e2b08b904576c,16150771dfd4776261284213b89c304e,entregue,2018-03-19,2018-03-20,2018-03-28,2018-03-29,2018-03-29,10,10,0,Sim,2025-11-13T00:17:18.905Z,11075,SANTOS,SP,2025-11-13T01:31:27.642Z,a416b6a846a11724393025641d4edd5e
00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,entregue,2017-04-26,2017-04-26,2017-05-04,2017-05-12,2017-05-15,16,19,-3,Sim,2025-11-13T00:17:18.905Z,15775,SANTA FE DO SUL,SP,2025-11-13T01:31:27.642Z,dd7ddc04e1b6c2c614352b383efe2d36
00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,entregue,2017-02-04,2017-02-04,2017-02-16,2017-03-01,2017-03-17,25,41,-16,Sim,2025-11-13T00:17:18.905Z,13226,VARZEA PAULISTA,SP,2025-11-13T01:31:27.642Z,df560393f3a51e74553ab94004ba5c87


root
 |-- id_pedido: string (nullable = true)
 |-- id_consumidor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- pedido_compra_timestamp: date (nullable = true)
 |-- pedido_aprovado_timestamp: date (nullable = true)
 |-- pedido_carregado_timestamp: date (nullable = true)
 |-- pedido_entregue_timestamp: date (nullable = true)
 |-- pedido_estimativa_entrega_timestamp: date (nullable = true)
 |-- tempo_entrega_dias: integer (nullable = true)
 |-- tempo_entrega_estimado_dias: integer (nullable = true)
 |-- diferenca_entrega_dias: integer (nullable = true)
 |-- entrega_no_prazo: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)
 |-- prefixo_cep: integer (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)
 |-- id_vendedor: string (nullable = true)



In [0]:
df_join_1.groupBy("entrega_no_prazo") \
    .count() \
    .orderBy("count", ascending=False) \
    .show()

+----------------+------+
|entrega_no_prazo| count|
+----------------+------+
|             Sim|102931|
|             Não|  7265|
|    Não Entregue|  3229|
+----------------+------+



In [0]:
qtd_nulos = df_join_1.filter(F.col("entrega_no_prazo").isNull()).count()
qtd_nulos

0

In [0]:
df_gold_ft_atrasos_pedidos_local_vendedor = df_join_1.select(
    F.col("id_pedido"),
    F.col("id_vendedor"),
    F.col("id_consumidor"),
    F.col("entrega_no_prazo"),
    F.col("tempo_entrega_dias"),
    F.col("tempo_entrega_estimado_dias"),
    F.col("cidade"),
    F.col("estado")
).withColumn("data_ingestao", F.current_timestamp())
display(df_gold_ft_atrasos_pedidos_local_vendedor.limit(10))
df_gold_ft_atrasos_pedidos_local_vendedor.printSchema()

id_pedido,id_vendedor,id_consumidor,entrega_no_prazo,tempo_entrega_dias,tempo_entrega_estimado_dias,cidade,estado,data_ingestao
00010242fe8c5a6d1ba2dd792cb16214,48436dade18ac8b2bce089ec2a041202,3ce436f183e68e07877b285a838db11a,Sim,7,16,CAMPOS DOS GOYTACAZES,RJ,2025-11-19T13:17:24.555Z
00018f77f2f0320c557190d7a144bdd3,dd7ddc04e1b6c2c614352b383efe2d36,f6dd3ec061db4e3987629fe6b26e5cce,Sim,16,19,SANTA FE DO SUL,SP,2025-11-19T13:17:24.555Z
000229ec398224ef6ca0657da4fc703e,5b51032eddd242adc84c38acab88f23d,6489ae5e4333f3693df5ad4372dab6d3,Sim,8,22,PARA DE MINAS,MG,2025-11-19T13:17:24.555Z
00024acbcdf0a6daa1e931b038114c75,9d7a1d34a5052409006425275ba1c2b4,d4eb9395c8c0431ee92fce09860c5a06,Sim,6,12,ATIBAIA,SP,2025-11-19T13:17:24.555Z
00042b26cf59d7ce69dfabb4e55b4fd9,df560393f3a51e74553ab94004ba5c87,58dbd0b2d70206bf40e62cd34e84d795,Sim,25,41,VARZEA PAULISTA,SP,2025-11-19T13:17:24.555Z
00048cc3ae777c65dbb7d2a0634bc1ea,6426d21aca402a131fc0a5d0960a3c90,816cbea969fe5b689b39cfc97a506742,Sim,7,22,UBERABA,MG,2025-11-19T13:17:24.555Z
00054e8431b9d7675808bcb819fb4a32,7040e82f899a04d1b434b795a43b4617,32e2e6ab09e778d99bf2e0ecd4898718,Sim,8,25,GUARARAPES,SP,2025-11-19T13:17:24.555Z
000576fe39319847cbb9d288c5617fa6,5996cddab893a4652a15592fb58ab8db,9ed5e522dd9dd85b4af4a077526d8117,Sim,5,21,PRAIA GRANDE,SP,2025-11-19T13:17:24.555Z
0005a1a1728c9d785b8e2b08b904576c,a416b6a846a11724393025641d4edd5e,16150771dfd4776261284213b89c304e,Sim,10,10,SANTOS,SP,2025-11-19T13:17:24.555Z
0005f50442cb953dcd1d21e1fb923495,ba143b05f0110f0dc71ad71b4466ce92,351d3cb2cee3c7fd0af6616c82df21d3,Sim,2,21,JANDIRA,SP,2025-11-19T13:17:24.555Z


root
 |-- id_pedido: string (nullable = true)
 |-- id_vendedor: string (nullable = true)
 |-- id_consumidor: string (nullable = true)
 |-- entrega_no_prazo: string (nullable = true)
 |-- tempo_entrega_dias: integer (nullable = true)
 |-- tempo_entrega_estimado_dias: integer (nullable = true)
 |-- cidade: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_ingestao: timestamp (nullable = false)



In [0]:
df_gold_ft_atrasos_pedidos_local_vendedor.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable('gold.ft_atrasos_pedidos_local_vendedor')


### 2.2 Criação das Views Analíticas

#### 2.2.1 `gold.view_tempo_medio_entrega_localidade`

In [0]:
view_name = "gold.view_tempo_medio_entrega_localidade"
source_table_name = "gold.ft_atrasos_pedidos_local_vendedor"

df_source = spark.table(source_table_name)

df_temp = (
    df_source
    .groupBy("cidade", "estado")
    .agg(
        F.round(F.avg("tempo_entrega_dias"), 2).alias("tempo_medio_entrega"),
        F.round(F.avg("tempo_entrega_estimado_dias"), 2).alias("tempo_medio_estimado")
    )
)

In [0]:
display(df_temp.limit(10))

cidade,estado,tempo_medio_entrega,tempo_medio_estimado
CAMPOS DOS GOYTACAZES,RJ,14.94,25.67
SANTA FE DO SUL,SP,11.53,24.0
PARA DE MINAS,MG,12.54,24.71
ATIBAIA,SP,8.92,19.44
VARZEA PAULISTA,SP,11.02,24.53
UBERABA,MG,11.04,25.53
GUARARAPES,SP,12.57,25.27
PRAIA GRANDE,SP,10.3,18.83
SANTOS,SP,9.04,18.86
JANDIRA,SP,8.31,20.17


In [0]:
df_view_tempo_medio_entrega_localidade= df_temp.withColumn(
    "entrega_maior_que_estimado",
    F.when(
        F.col("tempo_medio_entrega") > F.col("tempo_medio_estimado"), 
        F.lit("SIM")
    ).otherwise(F.lit("NÃO"))
)

In [0]:
display(df_view_tempo_medio_entrega_localidade.limit(10))

cidade,estado,tempo_medio_entrega,tempo_medio_estimado,entrega_maior_que_estimado
CAMPOS DOS GOYTACAZES,RJ,14.94,25.67,NÃO
SANTA FE DO SUL,SP,11.53,24.0,NÃO
PARA DE MINAS,MG,12.54,24.71,NÃO
ATIBAIA,SP,8.92,19.44,NÃO
VARZEA PAULISTA,SP,11.02,24.53,NÃO
UBERABA,MG,11.04,25.53,NÃO
GUARARAPES,SP,12.57,25.27,NÃO
PRAIA GRANDE,SP,10.3,18.83,NÃO
SANTOS,SP,9.04,18.86,NÃO
JANDIRA,SP,8.31,20.17,NÃO


In [0]:
df_view_tempo_medio_entrega_localidade.createOrReplaceTempView("gold.view_tempo_medio_entrega_localidade")

In [0]:
df_view_tempo_medio_entrega_localidade.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(view_name)

%md
####  2.2.2 `gold.view_vendedor_pontualidade`

In [0]:
view_name = "gold.view_vendedor_pontualidade"
source_table_name = "gold.ft_atrasos_pedidos_local_vendedor"

df_source = spark.table(source_table_name)

df_temp = (
    df_source
    .groupBy("id_vendedor")
    .agg(
        F.count("id_pedido").alias("total_pedidos"),
        F.sum(
            F.when(F.col("entrega_no_prazo") == "Não", 1).otherwise(0)
        ).alias("total_atrasados"),
    )
)


In [0]:
display(df_temp.limit(10))

id_vendedor,total_pedidos,total_atrasados
48436dade18ac8b2bce089ec2a041202,151,10
dd7ddc04e1b6c2c614352b383efe2d36,143,16
5b51032eddd242adc84c38acab88f23d,14,0
9d7a1d34a5052409006425275ba1c2b4,16,1
df560393f3a51e74553ab94004ba5c87,29,2
6426d21aca402a131fc0a5d0960a3c90,23,0
7040e82f899a04d1b434b795a43b4617,228,16
5996cddab893a4652a15592fb58ab8db,1,0
a416b6a846a11724393025641d4edd5e,181,19
ba143b05f0110f0dc71ad71b4466ce92,86,10


In [0]:
df_gold_view_vendedor_pontualidade = df_temp.withColumn(
    "percentual_atraso",
    F.round((F.col("total_atrasados") / F.col("total_pedidos") ),2 )
)
display(df_gold_view_vendedor_pontualidade.limit(10))

id_vendedor,total_pedidos,total_atrasados,percentual_atraso
48436dade18ac8b2bce089ec2a041202,151,10,0.07
dd7ddc04e1b6c2c614352b383efe2d36,143,16,0.11
5b51032eddd242adc84c38acab88f23d,14,0,0.0
9d7a1d34a5052409006425275ba1c2b4,16,1,0.06
df560393f3a51e74553ab94004ba5c87,29,2,0.07
6426d21aca402a131fc0a5d0960a3c90,23,0,0.0
7040e82f899a04d1b434b795a43b4617,228,16,0.07
5996cddab893a4652a15592fb58ab8db,1,0,0.0
a416b6a846a11724393025641d4edd5e,181,19,0.1
ba143b05f0110f0dc71ad71b4466ce92,86,10,0.12


In [0]:
df_gold_view_vendedor_pontualidade.createOrReplaceTempView("gold.view_vendedor_pontualidade")

In [0]:
df_gold_view_vendedor_pontualidade.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(view_name)

##  3º Projeto — Área Comercial

###  3.1 Criação da Dimensão de Tempo — `gold.dm_tempo`

In [0]:
df_range = df_pedido_total.select(
    F.min("data_pedido").alias("inicio"),
    F.max("data_pedido").alias("fim")
)
display(df_range)

inicio,fim
2016-09-04,2018-10-17


In [0]:
df_array_datas = df_range.select(
    F.sequence(F.col("inicio"), F.col("fim"), F.expr("interval 1 day")).alias("array_datas")
)
df_datas = df_array_datas.select(
    F.explode("array_datas").alias("sk_tempo")
)
display(df_datas.limit(10))

sk_tempo
2016-09-04
2016-09-05
2016-09-06
2016-09-07
2016-09-08
2016-09-09
2016-09-10
2016-09-11
2016-09-12
2016-09-13


In [0]:
df_dim_tempo = df_datas.select(
    F.col("sk_tempo"),
    F.year("sk_tempo").alias("ano"),
    F.quarter("sk_tempo").alias("trimestre"),
    F.month("sk_tempo").alias("mes"),
    F.weekofyear("sk_tempo").alias("semana_do_ano"),
    F.dayofmonth("sk_tempo").alias("dia"),
    F.dayofweek("sk_tempo").alias("dia_da_semana_num"), 
).withColumn(
    "mes_nome",
    F.when(F.col("mes") == 1, "Janeiro")
    .when(F.col("mes") == 2, "Fevereiro")
    .when(F.col("mes") == 3, "Março")
    .when(F.col("mes") == 4, "Abril")
    .when(F.col("mes") == 5, "Maio")
    .when(F.col("mes") == 6, "Junho")
    .when(F.col("mes") == 7, "Julho")
    .when(F.col("mes") == 8, "Agosto")
    .when(F.col("mes") == 9, "Setembro")
    .when(F.col("mes") == 10, "Outubro")
    .when(F.col("mes") == 11, "Novembro")
    .when(F.col("mes") == 12, "Dezembro")
) \
.withColumn(
    "dia_da_semana_nome",
    F.when(F.col("dia_da_semana_num") == 1, "Domingo")
    .when(F.col("dia_da_semana_num") == 2, "Segunda-feira")
    .when(F.col("dia_da_semana_num") == 3, "Terça-feira")
    .when(F.col("dia_da_semana_num") == 4, "Quarta-feira")
    .when(F.col("dia_da_semana_num") == 5, "Quinta-feira")
    .when(F.col("dia_da_semana_num") == 6, "Sexta-feira")
    .when(F.col("dia_da_semana_num") == 7, "Sábado")
) \
.withColumn(
    "eh_fim_de_semana",
    F.when(F.col("dia_da_semana_num").isin([1, 7]), "Sim").otherwise("Não")
)
display(df_dim_tempo.limit(5))
df_dim_tempo.printSchema()

sk_tempo,ano,trimestre,mes,semana_do_ano,dia,dia_da_semana_num,mes_nome,dia_da_semana_nome,eh_fim_de_semana
2016-09-04,2016,3,9,35,4,1,Setembro,Domingo,Sim
2016-09-05,2016,3,9,36,5,2,Setembro,Segunda-feira,Não
2016-09-06,2016,3,9,36,6,3,Setembro,Terça-feira,Não
2016-09-07,2016,3,9,36,7,4,Setembro,Quarta-feira,Não
2016-09-08,2016,3,9,36,8,5,Setembro,Quinta-feira,Não


root
 |-- sk_tempo: date (nullable = false)
 |-- ano: integer (nullable = false)
 |-- trimestre: integer (nullable = false)
 |-- mes: integer (nullable = false)
 |-- semana_do_ano: integer (nullable = false)
 |-- dia: integer (nullable = false)
 |-- dia_da_semana_num: integer (nullable = false)
 |-- mes_nome: string (nullable = true)
 |-- dia_da_semana_nome: string (nullable = true)
 |-- eh_fim_de_semana: string (nullable = false)



In [0]:
df_dim_tempo.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.dm_tempo")

### 3.2 Criação da Fato `gold.ft_vendas_geral`

In [0]:
df_avaliacoes = spark.table("silver.ft_avaliacoes_pedidos") \
                .groupBy("id_pedido") \
                .agg(F.avg("avaliacao").alias("avaliacao_media"))
display(df_avaliacoes.limit(5))

id_pedido,avaliacao_media
fc046d7776171871436844218f817d7d,5.0
d4665434b01caa9dc3e3e78b3eb3593e,5.0
e28abf2eb2f1fbcbdc2dd0cd9a561671,5.0
04fb47576993a3cb0c12d4b25eab6e4e,5.0
5f358d797a49fe2f24352f73426215f6,5.0


In [0]:
df_pedido_total.printSchema()
df_itens_pedidos.printSchema()
df_pedidos.printSchema()
df_cotacao = spark.table("silver.dm_cotacao_dolar")
df_cotacao.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- id_consumidor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- valor_total_pago_brl: double (nullable = true)
 |-- valor_total_pago_usd: double (nullable = true)
 |-- data_pedido: date (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

root
 |-- id_pedido: string (nullable = true)
 |-- id_item: integer (nullable = true)
 |-- id_produto: string (nullable = true)
 |-- id_vendedor: string (nullable = true)
 |-- preco_BRL: double (nullable = true)
 |-- preco_frete: double (nullable = true)
 |-- data_ingestao: timestamp (nullable = true)

root
 |-- id_pedido: string (nullable = true)
 |-- id_consumidor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- pedido_compra_timestamp: date (nullable = true)
 |-- pedido_aprovado_timestamp: date (nullable = true)
 |-- pedido_carregado_timestamp: date (nullable = true)
 |-- pedido_entregue_timestamp: date (nullable = true)
 |-- pedido_estimativa_en

In [0]:
df_pedidos_com_data = df_pedido_total.withColumn("data_join", F.to_date("data_pedido"))
df_cotacao_com_data = df_cotacao.withColumn("data_join", F.to_date("data")) 

df_join = df_itens_pedidos \
    .join(df_pedidos_com_data, on="id_pedido", how="inner") \
    .join(df_gold_ft_atrasos_pedidos_local_vendedor.select("id_pedido", "tempo_entrega_dias", "entrega_no_prazo"), on="id_pedido", how="left") \
    .join(df_avaliacoes, on="id_pedido", how="left") \
    .join(df_cotacao_com_data, on="data_join", how="left")

display(df_join.limit(5))

data_join,id_pedido,id_item,id_produto,id_vendedor,preco_BRL,preco_frete,data_ingestao,id_consumidor,status,valor_total_pago_brl,valor_total_pago_usd,data_pedido,data_ingestao,tempo_entrega_dias,entrega_no_prazo,avaliacao_media,data,cotacao_dolar,data_ingestao
2018-02-10,019886de8f385a39b75bedbb726fd4ef,1,e9a69340883a438c3f91739d14d3a56d,1b4c3a6f53068f0b6944d2d005c9fc89,159.9,28.5,2025-11-13T00:17:24.592Z,8cf88d7ba142365ef2ca619ef06f9a0f,entregue,188.4,57.41,2018-02-10,2025-11-13T02:23:03.057Z,13,Sim,5.0,2018-02-10,3.2815,2025-11-13T02:19:46.585Z
2017-11-30,06ff862a85c2402aa52dc9edf150bf30,1,4ce9ab528124f89e091b17d11aa2e97c,7e3f87d16fb353f408d467e74fbd8014,41.9,17.63,2025-11-13T00:17:24.592Z,0a978c825ff7d013133ddc7f77566172,entregue,59.53,18.26,2017-11-30,2025-11-13T02:23:03.057Z,28,Sim,3.0,2017-11-30,3.261,2025-11-13T02:19:46.585Z
2017-06-09,110ac0768c3e3a78e2937b1cb1ea3395,1,1b7ce992a80ac036dd9ab73d08289712,582d4f8675b945722eda7c0cb61ba4c7,25.5,15.1,2025-11-13T00:17:24.592Z,770d7945efb2aa13cbcf28eb7a353def,entregue,40.6,12.4,2017-06-09,2025-11-13T02:23:03.057Z,5,Sim,5.0,2017-06-09,3.2734,2025-11-13T02:19:46.585Z
2018-01-24,12dd6fe47b1a7c9e5742662223880dcc,1,63f4d40c05db6ade462cecef857eec34,fec6275253471ace26d209bbaa64cd0f,284.0,39.54,2025-11-13T00:17:24.592Z,95a911adbd9c6dee5b6ba9f764678fa3,entregue,323.54,101.22,2018-01-24,2025-11-13T02:23:03.057Z,14,Sim,5.0,2018-01-24,3.1964,2025-11-13T02:19:46.585Z
2017-05-13,139be8870b91e71fd70bb366305c8cde,1,4c1e109ecdf58453de365d217cefa64c,4e922959ae960d389249c378d1c939f5,120.0,13.18,2025-11-13T00:17:24.592Z,7f0f4bcffd7d9085eed3ad0a7814dd33,entregue,142.46,45.54,2017-05-13,2025-11-13T02:23:03.057Z,9,Sim,5.0,2017-05-13,3.1284,2025-11-13T02:19:46.585Z


In [0]:
df_gold_ft_vendas_geral = df_join.select(
    F.col("id_pedido"),
    F.col("id_item"),
    F.col("id_consumidor").alias("fk_cliente"),
    F.col("id_produto").alias("fk_produto"),
    F.col("id_vendedor").alias("fk_vendedor"),
    F.col("data_pedido").alias("fk_tempo"), 
    F.col("status").alias("status_pedido"),
    F.col("tempo_entrega_dias"),
    F.col("entrega_no_prazo"),
    
    F.col("preco_BRL").cast("decimal(12,2)").alias("valor_produto_brl"),
    F.col("preco_frete").cast("decimal(12,2)").alias("valor_frete_brl"),
    (F.col("preco_BRL") + F.col("preco_frete")).cast("decimal(12,2)").alias("valor_total_item_brl"),

    F.col("cotacao_dolar").cast("decimal(8,4)").alias("cotacao_dolar"),
    
    (F.col("preco_BRL") / F.col("cotacao_dolar")).cast("decimal(12,2)").alias("valor_produto_usd"),
    (F.col("preco_frete") / F.col("cotacao_dolar")).cast("decimal(12,2)").alias("valor_frete_usd"),
    ((F.col("preco_BRL") + F.col("preco_frete")) / F.col("cotacao_dolar")).cast("decimal(12,2)").alias("valor_total_item_usd"),

    F.col("avaliacao_media").cast("decimal(3,2)")
)
display(df_gold_ft_vendas_geral.limit(5))

id_pedido,id_item,fk_cliente,fk_produto,fk_vendedor,fk_tempo,status_pedido,tempo_entrega_dias,entrega_no_prazo,valor_produto_brl,valor_frete_brl,valor_total_item_brl,cotacao_dolar,valor_produto_usd,valor_frete_usd,valor_total_item_usd,avaliacao_media
019886de8f385a39b75bedbb726fd4ef,1,8cf88d7ba142365ef2ca619ef06f9a0f,e9a69340883a438c3f91739d14d3a56d,1b4c3a6f53068f0b6944d2d005c9fc89,2018-02-10,entregue,13,Sim,159.90,28.50,188.40,3.2815,48.73,8.69,57.41,5.00
06ff862a85c2402aa52dc9edf150bf30,1,0a978c825ff7d013133ddc7f77566172,4ce9ab528124f89e091b17d11aa2e97c,7e3f87d16fb353f408d467e74fbd8014,2017-11-30,entregue,28,Sim,41.90,17.63,59.53,3.2610,12.85,5.41,18.26,3.00
110ac0768c3e3a78e2937b1cb1ea3395,1,770d7945efb2aa13cbcf28eb7a353def,1b7ce992a80ac036dd9ab73d08289712,582d4f8675b945722eda7c0cb61ba4c7,2017-06-09,entregue,5,Sim,25.50,15.10,40.60,3.2734,7.79,4.61,12.40,5.00
12dd6fe47b1a7c9e5742662223880dcc,1,95a911adbd9c6dee5b6ba9f764678fa3,63f4d40c05db6ade462cecef857eec34,fec6275253471ace26d209bbaa64cd0f,2018-01-24,entregue,14,Sim,284.00,39.54,323.54,3.1964,88.85,12.37,101.22,5.00
139be8870b91e71fd70bb366305c8cde,1,7f0f4bcffd7d9085eed3ad0a7814dd33,4c1e109ecdf58453de365d217cefa64c,4e922959ae960d389249c378d1c939f5,2017-05-13,entregue,9,Sim,120.00,13.18,133.18,3.1284,38.36,4.21,42.57,5.00


In [0]:
df_gold_ft_vendas_geral.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("gold.ft_vendas_geral")

###  3.3 Criação da view `gold.view_vendas_por_periodo`

In [0]:
df_gold_ft_vendas_geral
df_dim_tempo
df_join = df_gold_ft_vendas_geral.join(
  df_dim_tempo,
  df_gold_ft_vendas_geral.fk_tempo == df_dim_tempo.sk_tempo,
  how = "inner"
)

display(df_join.limit(5))
df_join.printSchema()

id_pedido,id_item,fk_cliente,fk_produto,fk_vendedor,fk_tempo,status_pedido,tempo_entrega_dias,entrega_no_prazo,valor_produto_brl,valor_frete_brl,valor_total_item_brl,cotacao_dolar,valor_produto_usd,valor_frete_usd,valor_total_item_usd,avaliacao_media,sk_tempo,ano,trimestre,mes,semana_do_ano,dia,dia_da_semana_num,mes_nome,dia_da_semana_nome,eh_fim_de_semana
019886de8f385a39b75bedbb726fd4ef,1,8cf88d7ba142365ef2ca619ef06f9a0f,e9a69340883a438c3f91739d14d3a56d,1b4c3a6f53068f0b6944d2d005c9fc89,2018-02-10,entregue,13,Sim,159.90,28.50,188.40,3.2815,48.73,8.69,57.41,5.00,2018-02-10,2018,1,2,6,10,7,Fevereiro,Sábado,Sim
06ff862a85c2402aa52dc9edf150bf30,1,0a978c825ff7d013133ddc7f77566172,4ce9ab528124f89e091b17d11aa2e97c,7e3f87d16fb353f408d467e74fbd8014,2017-11-30,entregue,28,Sim,41.90,17.63,59.53,3.2610,12.85,5.41,18.26,3.00,2017-11-30,2017,4,11,48,30,5,Novembro,Quinta-feira,Não
110ac0768c3e3a78e2937b1cb1ea3395,1,770d7945efb2aa13cbcf28eb7a353def,1b7ce992a80ac036dd9ab73d08289712,582d4f8675b945722eda7c0cb61ba4c7,2017-06-09,entregue,5,Sim,25.50,15.10,40.60,3.2734,7.79,4.61,12.40,5.00,2017-06-09,2017,2,6,23,9,6,Junho,Sexta-feira,Não
12dd6fe47b1a7c9e5742662223880dcc,1,95a911adbd9c6dee5b6ba9f764678fa3,63f4d40c05db6ade462cecef857eec34,fec6275253471ace26d209bbaa64cd0f,2018-01-24,entregue,14,Sim,284.00,39.54,323.54,3.1964,88.85,12.37,101.22,5.00,2018-01-24,2018,1,1,4,24,4,Janeiro,Quarta-feira,Não
139be8870b91e71fd70bb366305c8cde,1,7f0f4bcffd7d9085eed3ad0a7814dd33,4c1e109ecdf58453de365d217cefa64c,4e922959ae960d389249c378d1c939f5,2017-05-13,entregue,9,Sim,120.00,13.18,133.18,3.1284,38.36,4.21,42.57,5.00,2017-05-13,2017,2,5,19,13,7,Maio,Sábado,Sim


root
 |-- id_pedido: string (nullable = true)
 |-- id_item: integer (nullable = true)
 |-- fk_cliente: string (nullable = true)
 |-- fk_produto: string (nullable = true)
 |-- fk_vendedor: string (nullable = true)
 |-- fk_tempo: date (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- tempo_entrega_dias: integer (nullable = true)
 |-- entrega_no_prazo: string (nullable = true)
 |-- valor_produto_brl: decimal(12,2) (nullable = true)
 |-- valor_frete_brl: decimal(12,2) (nullable = true)
 |-- valor_total_item_brl: decimal(12,2) (nullable = true)
 |-- cotacao_dolar: decimal(8,4) (nullable = true)
 |-- valor_produto_usd: decimal(12,2) (nullable = true)
 |-- valor_frete_usd: decimal(12,2) (nullable = true)
 |-- valor_total_item_usd: decimal(12,2) (nullable = true)
 |-- avaliacao_media: decimal(3,2) (nullable = true)
 |-- sk_tempo: date (nullable = false)
 |-- ano: integer (nullable = false)
 |-- trimestre: integer (nullable = false)
 |-- mes: integer (nullable = false)
 |-- se

In [0]:
df_view_temporal = df_join.groupBy(
    "ano", 
    "trimestre", 
    "mes", 
    "mes_nome", 
    "dia", 
    "dia_da_semana_num"
).agg(
    F.countDistinct("id_pedido").cast("bigint").alias("total_pedidos"),
    F.count("id_item").cast("bigint").alias("total_itens"),
    F.sum("valor_total_item_brl").cast("decimal(12,2)").alias("receita_total_brl"),
    F.sum("valor_total_item_usd").cast("decimal(12,2)").alias("receita_total_usd"),
    F.avg("valor_total_item_brl").cast("decimal(12,2)").alias("ticket_medio_brl"),
    F.avg("avaliacao_media").cast("decimal(3,2)").alias("avaliacao_media")
)
display(df_view_temporal.limit(5))
df_view_temporal.printSchema()

ano,trimestre,mes,mes_nome,dia,dia_da_semana_num,total_pedidos,total_itens,receita_total_brl,receita_total_usd,ticket_medio_brl,avaliacao_media
2017,1,1,Janeiro,10,3,6,6,1571.17,492.36,261.86,3.50
2017,1,2,Fevereiro,7,3,111,169,22089.55,7057.81,130.71,4.31
2018,2,6,Junho,18,2,246,393,51173.14,13632.88,130.21,4.65
2017,2,6,Junho,17,7,74,108,14798.33,4499.93,137.02,3.89
2018,2,4,Abril,18,4,280,419,57845.89,17095.04,138.06,3.75


root
 |-- ano: integer (nullable = false)
 |-- trimestre: integer (nullable = false)
 |-- mes: integer (nullable = false)
 |-- mes_nome: string (nullable = true)
 |-- dia: integer (nullable = false)
 |-- dia_da_semana_num: integer (nullable = false)
 |-- total_pedidos: long (nullable = false)
 |-- total_itens: long (nullable = false)
 |-- receita_total_brl: decimal(12,2) (nullable = true)
 |-- receita_total_usd: decimal(12,2) (nullable = true)
 |-- ticket_medio_brl: decimal(12,2) (nullable = true)
 |-- avaliacao_media: decimal(3,2) (nullable = true)



In [0]:
df_view_temporal.createOrReplaceTempView("gold.view_vendas_por_periodo")
df_view_temporal.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.view_vendas_por_periodo")

#### 3.3.1 Consultas Analíticas

Qual é o dia da semana com maior receita total em reais (`receita_total_brl`) ?

In [0]:
%sql
SELECT 
    dia_da_semana_num,
    CASE 
        WHEN dia_da_semana_num = 1 THEN 'Domingo'
        WHEN dia_da_semana_num = 2 THEN 'Segunda-feira'
        WHEN dia_da_semana_num = 3 THEN 'Terça-feira'
        WHEN dia_da_semana_num = 4 THEN 'Quarta-feira'
        WHEN dia_da_semana_num = 5 THEN 'Quinta-feira'
        WHEN dia_da_semana_num = 6 THEN 'Sexta-feira'
        WHEN dia_da_semana_num = 7 THEN 'Sábado'
    END AS nome_dia_semana,
    
    SUM(receita_total_brl) AS receita_acumulada

FROM 
    gold.view_vendas_por_periodo

GROUP BY 
    dia_da_semana_num

ORDER BY 
    receita_acumulada DESC

LIMIT 1;

dia_da_semana_num,nome_dia_semana,receita_acumulada
2,Segunda-feira,3313504.69


Considerando o último ano disponível na dimensão de tempo, qual foi o 
mês com maior ticket médio (`ticket_medio_brl`) ?

In [0]:
%sql
SELECT 
    ano,
    mes_nome,
    CAST(AVG(ticket_medio_brl) AS DECIMAL(12,2)) AS media_ticket_mensal

FROM 
    gold.view_vendas_por_periodo

WHERE 
    ano = (SELECT MAX(ano) FROM gold.view_vendas_por_periodo)

GROUP BY 
    ano, mes, mes_nome 

ORDER BY 
    media_ticket_mensal DESC

LIMIT 1;

ano,mes_nome,media_ticket_mensal
2018,Setembro,166.46


###  3.4. Criação da `gold.view_top_produto`

In [0]:
df_join = df_gold_ft_vendas_geral.join(
    df_produtos,
    df_gold_ft_vendas_geral.fk_produto == df_produtos.id_produto,
    how="left"
)

df_view_top_produto = df_join.groupBy(
    F.col("fk_produto").alias("id_produto"),
    F.col("categoria_produto")
).agg(
    F.count("id_item").cast("bigint").alias("quantidade_vendida"),
    F.countDistinct("id_pedido").cast("bigint").alias("total_pedidos"),

    F.sum("valor_produto_brl").cast("decimal(12,2)").alias("receita_brl"),
    F.sum("valor_produto_usd").cast("decimal(12,2)").alias("receita_usd"),
    
    F.avg("valor_produto_brl").cast("decimal(12,2)").alias("preco_medio_brl"),
    F.avg("avaliacao_media").cast("decimal(3,2)").alias("avaliacao_media"),
    F.avg("peso_produto_gramas").cast("decimal(8,2)").alias("peso_medio_gramas")
).orderBy(F.col("receita_brl").desc())

display(df_view_top_produto.limit(10))
df_view_top_produto.printSchema()

id_produto,categoria_produto,quantidade_vendida,total_pedidos,receita_brl,receita_usd,preco_medio_brl,avaliacao_media,peso_medio_gramas
5769ef0a239114ac3a854af00df129e4,telefonia_fixa,64,1,107520.00,33945.60,1680.00,1.00,750.00
bb50f2e236e5eea0100680137654686c,beleza_saude,238,187,78075.00,22403.29,328.05,4.26,400.00
422879e10f46682990de24d770e7f83d,ferramentas_jardim,1110,352,61808.79,18620.61,55.68,3.42,1550.00
6cdd53843498f92890544667809f1595,beleza_saude,173,151,60734.90,17965.20,351.07,4.24,900.00
d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,408,323,56486.14,17009.51,138.45,4.07,6550.00
a62e25e09e05e6faf31d90c6ec1aa3d1,relogios_presentes,513,172,54699.00,16196.22,106.63,3.03,1000.00
aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,751,431,53673.70,16096.07,71.47,3.91,2600.00
d5991653e037ccb7af6ed7d94246b249,informatica_acessorios,366,57,53384.13,16726.83,145.86,3.32,922.00
99a4788cb24856965c36a24e339b6058,cama_mesa_banho,595,467,52431.36,16039.72,88.12,3.85,1383.00
3dd2a17168ec895c781a9191c1e95ad7,informatica_acessorios,338,255,50676.20,14813.87,149.93,3.86,533.00


root
 |-- id_produto: string (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- quantidade_vendida: long (nullable = false)
 |-- total_pedidos: long (nullable = false)
 |-- receita_brl: decimal(12,2) (nullable = true)
 |-- receita_usd: decimal(12,2) (nullable = true)
 |-- preco_medio_brl: decimal(12,2) (nullable = true)
 |-- avaliacao_media: decimal(3,2) (nullable = true)
 |-- peso_medio_gramas: decimal(8,2) (nullable = true)



In [0]:
df_view_top_produto.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.view_top_produto")

In [0]:
%sql
    SELECT
        t.ano,
        t.mes,
        p.categoria_produto,
        v.id_pedido,
        v.id_item,
        v.valor_produto_brl,
        v.valor_produto_usd,
        v.avaliacao_media
    FROM
        gold.ft_vendas_geral v
    INNER JOIN
        gold.dm_tempo t ON v.fk_tempo = t.sk_tempo
    INNER JOIN
        silver.ft_produtos p ON v.fk_produto = p.id_produto
    WHERE
        LOWER(p.categoria_produto) LIKE 'fashion%'

ano,mes,categoria_produto,id_pedido,id_item,valor_produto_brl,valor_produto_usd,avaliacao_media
2017,12,fashion_bolsas_e_acessorios,31db96cf06ad051a3d234fc35ea2488a,1,49.00,14.94,5.00
2017,4,fashion_bolsas_e_acessorios,777ee9fd50fbe50e788ef536b1656b08,1,99.99,31.84,5.00
2018,1,fashion_bolsas_e_acessorios,ca0869c7a30172e146311fba95983c2a,1,29.99,9.31,4.00
2017,8,fashion_calcados,03952349aca310eeba691019501388de,1,84.90,26.79,4.00
2017,8,fashion_calcados,2f36e982eab2df4e0ca5890a0228907d,1,84.90,27.16,5.00
2017,11,fashion_bolsas_e_acessorios,3a119f52de09d1b2700815732def8b47,1,19.99,6.19,1.00
2017,11,fashion_calcados,6125b8598105f92aeac157727ff3bfc4,1,84.90,26.23,5.00
2018,8,fashion_bolsas_e_acessorios,2ccc4454c10457bb1cd9fb7465167f2f,1,49.90,12.86,2.00
2017,11,fashion_bolsas_e_acessorios,9fa3141e41678109d614cc3ee569cb98,1,129.99,40.25,1.00
2018,6,fashion_calcados,aa9db0502703dcac7b66401ee4128525,1,89.90,24.03,5.00


### 3.5 Criação da `view_vendas_produtos_esteticos`

In [0]:
query_fashion = """
CREATE OR REPLACE VIEW gold.view_vendas_produtos_esteticos AS

WITH dados_fashion AS (
    SELECT
        t.ano,
        t.mes,
        p.categoria_produto,
        v.id_pedido,
        v.id_item,
        v.valor_produto_brl,
        v.valor_produto_usd,
        v.avaliacao_media
    FROM
        gold.ft_vendas_geral v
    INNER JOIN
        gold.dm_tempo t ON v.fk_tempo = t.sk_tempo
    INNER JOIN
        silver.ft_produtos p ON v.fk_produto = p.id_produto
    WHERE
        LOWER(p.categoria_produto) LIKE 'fashion%'
),

agregado AS (
    SELECT
        ano,
        mes,
        categoria_produto,
        COUNT(DISTINCT id_pedido) AS total_pedidos,
        COUNT(*) AS total_itens_vendidos,
        
        -- Receita (Soma do valor dos produtos)
        SUM(valor_produto_brl) AS receita_total_brl,
        SUM(valor_produto_usd) AS receita_total_usd,
        
        -- Ticket Médio (Média do valor por item)
        AVG(valor_produto_brl) AS ticket_medio_brl,
        AVG(valor_produto_usd) AS ticket_medio_usd,
        
        -- Avaliação Média
        AVG(avaliacao_media) AS avaliacao_media
    FROM
        dados_fashion
    GROUP BY
        ano, mes, categoria_produto
)

SELECT
    ano,
    mes,
    categoria_produto,
    CAST(total_pedidos AS BIGINT),
    CAST(total_itens_vendidos AS BIGINT),
    CAST(receita_total_brl AS DECIMAL(12,2)),
    CAST(receita_total_usd AS DECIMAL(12,2)),
    CAST(ticket_medio_brl AS DECIMAL(12,2)),
    CAST(ticket_medio_usd AS DECIMAL(12,2)),
    CAST(avaliacao_media AS DECIMAL(3,2))
FROM
    agregado;
"""

spark.sql(query_fashion)

DataFrame[]

In [0]:
df_view_vendas_produtos_esteticos = spark.table("gold.view_vendas_produtos_esteticos")
display(df_view_vendas_produtos_esteticos.limit(10))
df_view_vendas_produtos_esteticos.printSchema()

ano,mes,categoria_produto,total_pedidos,total_itens_vendidos,receita_total_brl,receita_total_usd,ticket_medio_brl,ticket_medio_usd,avaliacao_media
2018,8,fashion_calcados,3,3,471.30,122.50,157.10,40.83,4.67
2018,3,fashion_calcados,12,12,1252.80,381.57,104.40,31.80,3.45
2018,2,fashion_bolsas_e_acessorios,102,120,11895.73,3663.09,99.13,30.53,4.17
2017,4,fashion_calcados,20,29,1881.90,596.87,64.89,20.58,4.75
2018,8,fashion_bolsas_e_acessorios,122,137,8666.03,2248.75,63.26,16.41,4.11
2017,3,fashion_roupa_masculina,7,13,743.50,239.23,57.19,18.40,4.15
2017,8,fashion_underwear_e_moda_praia,13,13,1111.00,351.99,85.46,27.08,4.38
2017,8,fashion_roupa_infanto_juvenil,1,1,49.90,15.98,49.90,15.98,1.00
2018,7,fashion_bolsas_e_acessorios,76,79,6202.77,1633.22,78.52,20.67,4.57
2017,12,fashion_roupa_masculina,4,4,450.89,137.44,112.72,34.36,4.50


root
 |-- ano: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- categoria_produto: string (nullable = true)
 |-- total_pedidos: long (nullable = false)
 |-- total_itens_vendidos: long (nullable = false)
 |-- receita_total_brl: decimal(12,2) (nullable = true)
 |-- receita_total_usd: decimal(12,2) (nullable = true)
 |-- ticket_medio_brl: decimal(12,2) (nullable = true)
 |-- ticket_medio_usd: decimal(12,2) (nullable = true)
 |-- avaliacao_media: decimal(3,2) (nullable = true)



In [0]:
%sql
SHOW TABLES IN medalhao.gold;

database,tableName,isTemporary
gold,dm_tempo,false
gold,ft_atrasos_pedidos_local_vendedor,false
gold,ft_atrasos_pedidos_localidade,false
gold,ft_vendas_consumidor_local,false
gold,ft_vendas_geral,false
gold,view_tempo_medio_entrega_localidade,false
gold,view_top_produto,false
gold,view_total_compras_por_consumidor,false
gold,view_vendas_localidade,false
gold,view_vendas_localidade_geral,false
